Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Lars Janssen"

---

For those not familiar with Python, a quick overview is given [here](https://github.com/palcu/python-for-competitive-programming/blob/master/python-for-competitive-programming.ipynb).

# Notebook BAPC week 9: Dynamic Programming I

In [2]:
from io import StringIO
from sys import stdin
# Overwrite the jupyter input function.
def input():
    return stdin.readline()

## Cent Savings

Because Dynamic Programming is mostly about practice, this notebook will be a little shorter than usual. We will produce a solution for the Kattis problem [Cent Savings](https://open.kattis.com/problems/centsavings), first using top-down DP (with an array instead of a dictionary) and afterwards using bottom-up DP. Read the problem.

### A rounding function
One of the very first things we will need is a function `round10` to round positive integers to the nearest multiple of 10. Finish the code in the cell below.

In [1]:
def round10(n):
    return int((n+5)/10)*10

In [6]:
assert round10(94) == 90
assert round10(95) == 100

### The first sample input
Given 5 items with costs `[13, 21, 55, 60, 42]` and a single divider, the minimum total cost is 190. In fact, all solutions yield a total cost of 190:

In [ ]:
items = [13, 21, 55, 60, 42]
for i in range(1, 5):
    assert 190 == round10(sum(items[:i])) + round10(sum(items[i:]))

### A first solution using recursion
If we let `cost(n,d)` be the minimum total cost of buying the first `n` items with `d` dividers, we can make a few initial observations:
* `cost(0,d) = 0` trivially.
* `cost(n,0) = round10(sum(items[:n]))`, because there is no divider to place and we just have to "ring up" the first `n` items.
* Our answer will be `cost(N, D)`.

Our main task will be to compute `cost(n, d)` given the result of subproblems. The key observation here is the following:
* If we know `cost(k, d-1)` for some `k < n`, placing a divider between items `k` and `k+1` yields a total cost of `cost(k, d-1)` plus the cost of all items from `k+1` to `n`.
* By minimizing the above over all `k < n`, we get `cost(n, d)`.

This allows us to produce our first recursive solution. Finish the code below.

In [56]:
def centsavings_recur(items, D):
    def cost(n, d):
        # In this nested function, the variables `N` and `items` are available.
        if n == 0:
            return 0
        if d == 0:
            return round10(sum(items[:n]))
        bestprice = round10(sum(items[:n]))
        for k in range(0,n):
            bestprice = min(bestprice, cost(k, d-1) + round10(sum(items[k:n])))
        return bestprice
                
    N = len(items)
    return cost(N, D)

In [59]:
assert centsavings_recur([13, 21, 55, 60, 42], 1) == 190
assert centsavings_recur([1, 1, 1, 1, 1], 2) == 0
for i in range(1, 10):
    assert centsavings_recur([10] * 10, i) == 100
assert centsavings_recur([5, 6, 5, 5, 7, 6, 1], 2) == 30

%time centsavings_recur([10] * 20, 10) == 200
#%time centsavings_recur([10] * 30, i) == 300  # This one will take an awfully long time to finish.

CPU times: user 430 ms, sys: 0 ns, total: 430 ms
Wall time: 429 ms


True

### A second solution using top-down DP
The above code will produce the right solution, however it is too slow. We are not yet exploiting the "overlapping subproblems" property that makes DP problems efficient to solve. We will add a memoization array -- a memoization dictionary will be too slow for this problem. Finish the code below. It should be a minimal change relative to `centsavings_recur`.

In [112]:
def centsavings_memo(items, D):
    def cost(n, d):
        # In this nested function, the variables `memo`, `N` and `items` are available.
        if memo[n][d] == 10**9:
            if n == 0:
                return 0
            if d == 0:
                return round10(sum(items[:n]))
            # Be sure to *set* memo[n][d]!
            best = round10(sum(items[:n]))
            for k in range(0,n):
                best = min(best, cost(k, d-1) + round10(sum(items[k:n])))
            memo[n][d] = best
            return best
        return memo[n][d]
    
    N = len(items)
    memo = [[10**9 for _ in range(D+1)] for _ in range(N+1)]
    return cost(N, D)

In [113]:
assert centsavings_memo([13, 21, 55, 60, 42], 1) == 190, "Did you remember to set memo[n][d]?"
assert centsavings_memo([1, 1, 1, 1, 1], 2) == 0
for i in range(1, 10):
    assert centsavings_memo([10] * 10, i) == 100
assert centsavings_memo([5, 6, 5, 5, 7, 6, 1], 2) == 30

# Note: the first time you run this cell, this code may install an extra Python package on your computer.
import sys
!{sys.executable} -m pip install stopit
import stopit

with stopit.ThreadingTimeout(3.0) as t:
    assert centsavings_memo([10] * 200, 20) == 2000
assert t.state == t.EXECUTED, "Your code is too slow. The change should be minimal w.r.t `centsavings_recur`!"

### A third solution using top-down DP and cumulative sums
The previous code again produces the right solution, but its runtime of $\mathcal O(N^3 D)$ is still too slow to work for the largest input size $N=2000, D=20$. The problem here is that we keep recomputing the same *range-sums* `sum(items[k:n])` many times. We can circumvent this problem by observing that
`sum(items[k:n]) == sum(items[:n]) - sum(items[:k])`. Therefore, if we precompute the *cumulative sums* `sum(items[:k])`, we may extract the range-sums for free. Use this insight to finish the code below.

In [213]:
def centsavings_td(items, D):
    def cost(n, d):
        # In this nested function, the variables `cumsums`, `memo`, `N` and `items` are available.
        if memo[n][d] == 10**9:
            # The code below should be a minimal change to `centsavings_memo`.
            # YOUR CODE HERE
            if n == 0:
                return 0
            if d == 0:
                return round10(cumsums[n])
            # Be sure to *set* memo[n][d]!
            best = round10(cumsums[-1])
            for k in range(0,n):
                best = min(best, cost(k, d-1) + round10(cumsums[n] - cumsums[k]))
            memo[n][d] = best
            return best
        return memo[n][d]
    
    N = len(items)
    cumsums = [0 for _ in range(N+1)]
    # Fill the cumsums list.
    # YOUR CODE HERE
    for k in range(0,N+1):
        cumsums[k] = sum(items[:k])
    assert cumsums[-1] == sum(items), "Did you fill the `cumsums` list until its final index?"
    memo = [[10**9 for _ in range(D+1)] for _ in range(N+1)]
    print(cost(N,D))
    return cost(N, D)

In [215]:
assert centsavings_td([13, 21, 55, 60, 42], 1) == 190
assert centsavings_td([1, 1, 1, 1, 1], 2) == 0
for i in range(1, 10):
    assert centsavings_td([10] * 10, i) == 100
assert centsavings_td([5, 6, 5, 5, 7, 6, 1], 2) == 30

# Note: the first time you run this cell, this code may install an extra Python package on your computer.
import sys
!{sys.executable} -m pip install stopit
import stopit

with stopit.ThreadingTimeout(3.0) as t:
    assert centsavings_td([1] * 2000, 2) == 1990
assert t.state == t.EXECUTED, "Your code is too slow. Did you use the `cumsums` variable everywhere you could, also in the case `d=0`?"

### A forth solution using bottom-up DP and cumulative sums
Instead of recurring top-down and exploiting the memoization array for the subproblems, we can also start bottom-up and build the solution as we go. Finish the code below.

In [14]:
def centsavings_bu(items, D):
    N = len(items)
    cumsums = [0 for _ in range(N+1)]
    # Fill the cumsums list.
    # YOUR CODE HERE
    for k in range(1,N+1):
        cumsums[k] = cumsums[k-1] + items[k-1]
    assert cumsums[-1] == sum(items), "Did you fill the `cumsums` list until its final index?"
    
    cost = [[10**9 for _ in range(D+1)] for _ in range(N+1)]
    # Initialize the base cases.
    for d in range(D+1):
        cost[0][d] = 0
    for n in range(N+1):
        cost[n][0] = round10(cumsums[n])
        
    # Build the DP matrix bottom-up.
    for d in range(1, D+1):
        for n in range(1, N+1):
            for k in range(0,n):
                cost[n][d] = min(cost[n][d], cost[k][d-1] + round10(cumsums[n] - cumsums[k]))
    return cost[n][d]

In [15]:
assert centsavings_bu([13, 21, 55, 60, 42], 1) == 190
assert centsavings_bu([1, 1, 1, 1, 1], 2) == 0
for i in range(1, 10):
    assert centsavings_bu([10] * 10, i) == 100
assert centsavings_bu([5, 6, 5, 5, 7, 6, 1], 2) == 30

# Note: the first time you run this cell, this code may install an extra Python package on your computer.
import sys
!{sys.executable} -m pip install stopit
import stopit

with stopit.ThreadingTimeout(3.0) as t:
    assert centsavings_bu([10] * 200, 20) == 2000
assert t.state == t.EXECUTED, "Your code is too slow. :-("

with stopit.ThreadingTimeout(3.0) as t:
    assert centsavings_bu([1] * 2000, 2) == 1990
assert t.state == t.EXECUTED, "Your code is too slow. :-("

### Final assignment
Now choose either the bottom-up or the top-down solution and send it to Kattis to grab that "Accepted"!